In [1]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [3]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 5 columns):
song_owner      6304 non-null object
song_title      6304 non-null object
song_element    6304 non-null object
verse           6304 non-null object
artist          6304 non-null object
dtypes: object(5)
memory usage: 246.3+ KB


In [5]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Kanye West']

In [6]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1933 entries, 913 to 6303
Data columns (total 5 columns):
song_owner      1933 non-null object
song_title      1933 non-null object
song_element    1933 non-null object
verse           1933 non-null object
artist          1933 non-null object
dtypes: object(5)
memory usage: 90.6+ KB


In [7]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [8]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
913,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],"We know, we know we got it, We know, we know w...",Kanye West,"[we, know, we, know, we, got, it, we, know, we..."
1281,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],"We know, we know we got it, We know, we know w...",Kanye West,"[we, know, we, know, we, got, it, we, know, we..."
1746,Drake,Pop Style (Single Version),[Verse 2: Kanye West],"They like, ""Pablo—"", ""Why are all the windows ...",Kanye West,"[they, like, ""pablo—"", ""why, are, all, the, wi..."
1931,Drake,Glow,[Chorus: Kanye West],"Watch out for me, I'm bound to glow, You bette...",Kanye West,"[watch, out, for, me, i'm, bound, to, glow, yo..."
1932,Drake,Glow,[Verse 1: Kanye West],"Your boy all glowed up, Every time we touch do...",Kanye West,"[your, boy, all, glowed, up, every, time, we, ..."


In [9]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [10]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
913,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],"We know, we know we got it, We know, we know w...",Kanye West,"[we, know, we, know, we, got, it, we, know, we...","[we, know, ,, we, know, we, got, it, ,, we, kn..."
1281,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],"We know, we know we got it, We know, we know w...",Kanye West,"[we, know, we, know, we, got, it, we, know, we...","[we, know, ,, we, know, we, got, it, ,, we, kn..."
1746,Drake,Pop Style (Single Version),[Verse 2: Kanye West],"They like, ""Pablo—"", ""Why are all the windows ...",Kanye West,"[they, like, ""pablo—"", ""why, are, all, the, wi...","[they, like, ,, ``, pablo—, '', ,, ``, why, ar..."
1931,Drake,Glow,[Chorus: Kanye West],"Watch out for me, I'm bound to glow, You bette...",Kanye West,"[watch, out, for, me, i'm, bound, to, glow, yo...","[watch, out, for, me, ,, i, 'm, bound, to, glo..."
1932,Drake,Glow,[Verse 1: Kanye West],"Your boy all glowed up, Every time we touch do...",Kanye West,"[your, boy, all, glowed, up, every, time, we, ...","[your, boy, all, glowed, up, ,, every, time, w..."


In [11]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [12]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

913     we know we know we got it we know we know we g...
1281    we know we know we got it we know we know we g...
1746    they like pablo why are all the windows tinted...
1931    watch out for me im bound to glow you better w...
1932    your boy all glowed up every time we touch dow...
Name: verse, dtype: object

In [13]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
913,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],we know we know we got it we know we know we g...,Kanye West,"[we, know, we, know, we, got, it, we, know, we...","[we, know, ,, we, know, we, got, it, ,, we, kn..."
1281,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],we know we know we got it we know we know we g...,Kanye West,"[we, know, we, know, we, got, it, we, know, we...","[we, know, ,, we, know, we, got, it, ,, we, kn..."
1746,Drake,Pop Style (Single Version),[Verse 2: Kanye West],they like pablo why are all the windows tinted...,Kanye West,"[they, like, ""pablo—"", ""why, are, all, the, wi...","[they, like, ,, ``, pablo—, '', ,, ``, why, ar..."
1931,Drake,Glow,[Chorus: Kanye West],watch out for me im bound to glow you better w...,Kanye West,"[watch, out, for, me, i'm, bound, to, glow, yo...","[watch, out, for, me, ,, i, 'm, bound, to, glo..."
1932,Drake,Glow,[Verse 1: Kanye West],your boy all glowed up every time we touch dow...,Kanye West,"[your, boy, all, glowed, up, every, time, we, ...","[your, boy, all, glowed, up, ,, every, time, w..."


In [14]:
def get_words(df):
    all_words = []
    
    for index, row in df['verse'].iteritems():
        new_words = re.findall(r"\b[a-z']+\b", unidecode(row))
        all_words += new_words
        
    return all_words

In [15]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
913,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],we know we know we got it we know we know we g...,Kanye West,"[we, know, we, know, we, got, it, we, know, we...","[we, know, ,, we, know, we, got, it, ,, we, kn..."
1281,Chance The Rapper,All We Got,[Outro: Kanye West & Chicago Children's Choir],we know we know we got it we know we know we g...,Kanye West,"[we, know, we, know, we, got, it, we, know, we...","[we, know, ,, we, know, we, got, it, ,, we, kn..."
1746,Drake,Pop Style (Single Version),[Verse 2: Kanye West],they like pablo why are all the windows tinted...,Kanye West,"[they, like, ""pablo—"", ""why, are, all, the, wi...","[they, like, ,, ``, pablo—, '', ,, ``, why, ar..."
1931,Drake,Glow,[Chorus: Kanye West],watch out for me im bound to glow you better w...,Kanye West,"[watch, out, for, me, i'm, bound, to, glow, yo...","[watch, out, for, me, ,, i, 'm, bound, to, glo..."
1932,Drake,Glow,[Verse 1: Kanye West],your boy all glowed up every time we touch dow...,Kanye West,"[your, boy, all, glowed, up, every, time, we, ...","[your, boy, all, glowed, up, ,, every, time, w..."


In [16]:
get_words(model_df)

['we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'music',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'music',
 'music',
 'is',
 'all',
 'we',
 'got',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'music',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'we',
 'know',
 'we',
 'know',
 'we',
 'got',
 'it',
 'music',
 'music',
 'is',
 'all',
 'we',
 'got',
 'they',
 'like'

In [17]:
verses = list(model_df.verse.values)

In [18]:
verses[:10]

['we know we know we got it we know we know we got it we know we know we got it we know we know we got it music we know we know we got it we know we know we got it we know we know we got it we know we know we got it music music is all we got',
 'we know we know we got it we know we know we got it we know we know we got it we know we know we got it music we know we know we got it we know we know we got it we know we know we got it we know we know we got it music music is all we got',
 'they like pablo why are all the windows tinted on your tahoe why do you know every single bitch that i know why cant you just shut your mouth and take the high road fuck if i know thats that chicago south south side thats the motto cop a crib and spend ten million on remodel take the devils out my life and preach the gospel cause i know we went way way past the line of scrimmage throne is back up in it in the field like emmitt yall get so offended i be blacking out i aint backing out jay about his busines

In [19]:
def make_word_dictionary(verses):
    following = {}
    count = {}
    for verse in verses:
        words = verse.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
                
    return count, following

In [20]:
count, following = make_word_dictionary(verses)

In [21]:
def create_bigrams(verses):
    bigrams = {}
    for words in verses:
        words = words.split()
        words = [x for x in words if x]
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)   
    return bigrams

In [22]:
verses_bigrams = create_bigrams(verses)

In [23]:
def generate_text_from_bigram(following, bigram, num_words, seeds):    
    all_bigrams = list(bigram.keys())
    current_bigram = random.choice(seeds)
    text = [current_bigram]
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            
            next_word = random.choice(bigram[current_bigram])
        else:
            if following[text[-1]]:
                next_word = random.choice(following[text[-1]])
            else:
                continue
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)

In [24]:
seeds = ["bitch", "love", "drugs", "cash", "guns", "club", "friends", "paper", "liquor", "apple",
         "blue", "red", "streets", "people", "brother", "cap"]

generated_raps = []
for _ in range(0,100):
    wordlist = generate_text_from_bigram(following, verses_bigrams, 33, seeds)
    generated_raps.append(wordlist)
    print(wordlist)
    print('\n')

people to fifteen hundred in a tight dress dancin close to snipe you truth told i finished school and she tryin im arguin like what you lays on me she hate white cock


guns from the rap and make you juke like walter payton so grand theft auto come up tryna get this cash with my clique so here since theyll probably put this online let


cash let me know let me fix the check what is if you dont know what happen tisha your boyfriend since he is three years old and truth betold im fucking on your


cash the sky hey maybe so all of the motherfucking building but do you all it all it all behind kanye step away from me if i always say kanye you aint let


paper look like emitt till she was just so tense is but im still myself and i changed my stars i went and found out it wasnt tryna get this cash with my


apple and e p jay boogie right chief keef king louie this is my new chick so we gon do it by myself lookin for a chance at loving you way i can


cash the hell i test the code four times 

liquor store ah and i had to calm her down memory lane you know what i would go into the vocal booth roll a joint smoke a little champagne cranberry vodka feelin lied


brother was cellphone back then then if you know we know we had packed a crowd big brother kanye west yup swag king cole yup dj khaled just say good night i get


guns the paparazzi damn these niggas wear suits cause you do shit the pyramids shall rise 30 hours look at me you re getting at age my grandfather s funeral like three weeks


paper look like halle berry posters in the industry if nobodys real then i would fall i m not into all that bruh i been thinking and it got girls callin me babycakes


friends my blessings cause you was me youd be dead or in jail all these long ass intro you got it what what baba baa one a million in production you i done


drugs for it you know one thing when i awoke i spent that bread she need to see me again again again i will smile and next time im on my own lil


streets playa thats

In [25]:
columns = ['markov_generated_verses']

kanye_markov_df = pd.DataFrame(generated_raps, columns=columns)

In [26]:
kanye_markov_df['artist'] = 'Kanye West'

kanye_markov_df

,markov_generated_verses,artist
0,people to fifteen hundred in a tight dress dan...,Kanye West
1,guns from the rap and make you juke like walte...,Kanye West
2,cash let me know let me fix the check what is ...,Kanye West
3,cash the sky hey maybe so all of the motherfuc...,Kanye West
4,paper look like emitt till she was just so ten...,Kanye West
5,apple and e p jay boogie right chief keef king...,Kanye West
6,cash the hell i test the code four times dont ...,Kanye West
7,guns sex with kickin out trickin on models dim...,Kanye West
8,brother was drinking earlier now i need you no...,Kanye West
9,guns the mall don t care where you from the al...,Kanye West


In [27]:
with open('data/kanye_markov_raps.pkl', 'wb') as picklefile:
    pickle.dump(kanye_markov_df, picklefile)